## To do
- Automate deletion of outbox and google form responses (in form document)
- Remove telephone number option from form
- Create logs
- Automate authentication process
- api limits?
- test test test



## When image manipulation doesn't work
- if image wrong size but otherwise good play with resize variable ( currrently set to .75-.8, max is 1),
- if no signature or all black consider changing colour threshold for image binary (currently set to 80) see line: ret,bw_array = cv.threshold(signature,80,255,cv.THRESH_BINARY) in postal_vote.py
- if image crops incorrectly (normally too big) look at tweaking dbscan cluster requirments... currently DBSCAN(eps = 3, min_samples = 5) in postal_vote.py

In [2]:
#data has been submitted with google form, photo too

#use google drive api to connect to google form
#https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#https://pythonhosted.org/PyDrive/quickstart.html
#thse libraries are for connecting to google drive for the image
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth


#for sending the email
import smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#for writing/deleting from local computer
import os
import time


#functions to populate postal form and crop, insert image
%run -i 'postal_vote.py'

In [ ]:
#two factor verification must be disabled, low security options enabled
#use very long password!!!!
#when email errors throw try nvigating to gmail account to address access errors

smtp_server = "smtp.gmail.com"
port = 587  # For start tls
sender_email = "postalvoteapplication@gmail.com"
password =  #????

In [ ]:
from pydrive.auth import GoogleAuth

#this method requires a click a browser to authenticate
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)

In [ ]:
#potential work around to automate verification process, worked sometimes, delete mycreds.txt file if it doesn't work
#try increasing time credentials are valid for
#ensure other google accounts dont auto log in

# # Try to load saved client credentials
# gauth.LoadCredentialsFile("mycreds.txt")
# if gauth.credentials is None:
#     # Authenticate if they're not there
#     gauth.LocalWebserverAuth()
# elif gauth.access_token_expired:
#     # Refresh them if expired
#     gauth.Refresh()
# else:
#     # Initialize the saved creds
#     gauth.Authorize()
# # Save the current credentials to a file
# gauth.SaveCredentialsFile("mycreds.txt")

# drive = GoogleDrive(gauth)

In [ ]:
# use creds to create a client to interact with the Google Drive API
#this is the sheets api for the form responses (that sync to spreadsheet)

#still unsure if api limits will be in issue
scope = ['https://www.googleapis.com/auth/spreadsheets', "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("postal_responses").sheet1

# Extract all records
records = sheet.get_all_records()

In [ ]:
records = sheet.get_all_records()
records

In [ ]:
# Extract all records
#work in progress

# add loop, run and wait for errors :)
#for i in range(1000):
 #   time.sleep(30)


records = sheet.get_all_records()

for r in records:
    first_name = r['First names']
    surname = r['Surname'] 
    home_address1 = r['Home address line 1'] 
    home_address2 = r['Home address line 2'] 
    home_address3 = r['Home address line 3'] #
    postcode = r['Postcode']
    date_of_birth = r['Date of birth'].replace('/','')
    telephone = str(r['Telephone (optional)'])


    email_send = r['Email address'] 
    if r['Would you like to include your email address in your postal vote application form?']=='Yes':
        email = r['Email address']
    else:
        email = ''

    if r['How long do you want a postal vote for?']=='Until further notice (permanent postal vote)':
        next_election_only = True
    else:
        next_election_only = False

    #if sending ballot to different address from home address
    #set to empty strings if not needed
    ballot_address1 = r['Ballot address line 1'] #limit length 30
    ballot_address2 = r['Ballot address line 2'] #limit length 30
    ballot_address3 = r['Ballot address line 3']#limit length 30
    ballot_postcode = r['Ballot postcode']
    ballot_reason = r["The reason you'd like your ballot sent to a different address"] #limit length 90


    #resize siganture, must not exceed 1 (1=max dimensions possible)
    #worth testing different resizes out
    #feature for end user?
    resize = .8

    #Assets included in repo
    #Using different assets may prevent functions from working as expected
    postal_path = "postal_vote_application_form.jpg"
    font_path = "arial.ttf"

    #id of photo in google drive
    photo_id = r['Signature photo'].split('id=')[1]

    #use google drive api to call signature photo
    try:
        signature = drive.CreateFile({'id': photo_id})

    except:
        print('BREAK')
        break

    #save file
    signature.GetContentFile('signature/signature{}.jpg'.format(photo_id))


    #load signature as array
    signature_path = 'signature/signature{}.jpg'.format(photo_id)


    _test1 = postal_vote(first_name, surname, postcode, date_of_birth,
                    home_address1, home_address2, home_address3,telephone, email,
                    ballot_address1, ballot_address2, ballot_address3,
                    ballot_postcode, ballot_reason, next_election_only,
                    postal_path, font_path)

    _test2= add_signature(_test1, signature_path, resize)

    export_as_pdf('processed/application{}'.format(photo_id), _test2)



    receiver_email = email_send
    subject = "Postal vote application form"
    body = "This is an email with attachment sent from Python"

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email  # Recommended for mass emails

    message.attach(MIMEText(body, "plain"))

    filename = "processed/application{}.pdf".format(photo_id)  # In same directory as script

    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Create a secure SSL context
    context = ssl.create_default_context()

    # Try to log in to server and send email
    try:
        server = smtplib.SMTP(smtp_server,port)
        server.ehlo() # Can be omitted
        server.starttls(context=context) # Secure the connection
        server.ehlo() # Can be omitted
        server.login(sender_email, password)
        # TODO: Send email here
        server.sendmail(sender_email, receiver_email, text)
    except Exception as e:
        # Print any error messages to stdout
        print(e)
    finally:
        server.quit()


    #DELETE RECORDS
    #also need to delete sent emails and form responses
    #trash drive file
    # signature.Trash()  # Move file to trash.
    # signature.UnTrash()  # Move file out of trash.

    #turn on during live deployment
    signature.Delete()  # Permanently delete the file.
    sheet.delete_row(2) #row 1 is header, deletes top row
    os.remove("processed/application{}.pdf".format(photo_id))
    os.remove('signature/signature{}.jpg'.format(photo_id))
    print ("Form processed, id: {}".format(photo_id))
    time.sleep(15)

In [ ]:
#signature.Delete()  # Permanently delete the file.
    
sheet.delete_row(2) #row 1 is header, deletes top row

In [3]:
time.sleep(15)